In [15]:
from tensorflow.keras import layers, Model, models, Sequential

class Inception(layers.Layer):
    def __init__(self, ch1x1, ch3x3red, ch3x3, ch5x5red, ch5x5, pool_proj, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # 定義四個分支
        # 第一個分支為 1x1 的卷積
        self.branch1 = layers.Conv2D(ch1x1, kernel_size=1, activation="relu")
        # 第二個分支為 1x1 的卷積 + 3x3的卷積 (輸出的大小 = 輸入的大小)
        self.branch2 = Sequential([
            layers.Conv2D(ch3x3red, kernel_size=1, activation="relu"),
            layers.Conv2D(ch3x3, kernel_size=3, padding="SAME", activation="relu")])     
        # 第三個分支為 1x1 的卷積 + 5x5的卷積 (輸出的大小 = 輸入的大小)
        self.branch3 = Sequential([
            layers.Conv2D(ch5x5red, kernel_size=1, activation="relu"),
            layers.Conv2D(ch5x5, kernel_size=5, padding="SAME", activation="relu")])      
        # 第四個分支為 3x3 的最大池化 + 1x1的卷積  (輸出的大小 = 輸入的大小)
        self.branch4 = Sequential([
            layers.MaxPool2D(pool_size=3, strides=1, padding="SAME"),  
            layers.Conv2D(pool_proj, kernel_size=1, activation="relu")])                  

    def call(self, inputs, **kwargs):
        branch1 = self.branch1(inputs)   # 輸入的特徵圖分別進入分支 1 至 4
        branch2 = self.branch2(inputs)
        branch3 = self.branch3(inputs)
        branch4 = self.branch4(inputs)
        outputs = layers.concatenate([branch1, branch2, branch3, branch4])
        return outputs

In [16]:
# 輔助分類器
class InceptionAux(layers.Layer):
    def __init__(self, num_classes, **kwargs):
        super(InceptionAux, self).__init__(**kwargs)
        # 第一層 :平均池化層
        self.averagePool = layers.AvgPool2D(pool_size=5, strides=3)
        # 第二層 :1x1卷積層
        self.conv = layers.Conv2D(128, kernel_size=1, activation="relu")
        # 第三層 :全連接層
        self.fc1 = layers.Dense(1024, activation="relu")
        # 第四層 :全連接層, 節點個數是非類數目
        self.fc2 = layers.Dense(num_classes)
        # 第五層 :softmax
        self.softmax = layers.Softmax()

    def call(self, inputs, **kwargs):
        x = self.averagePool(inputs)
        x = self.conv(x)
        x = layers.Flatten()(x)
        x = layers.Dropout(rate=0.5)(x)
        x = self.fc1(x)
        x = layers.Dropout(rate=0.5)(x)
        x = self.fc2(x)
        x = self.softmax(x)

        return x

In [17]:
def GoogLeNet(im_height=224, im_width=224, class_num=1000, aux_logits=False):
    # tensorflow中的tensor通道排序是NHWC
    input_image = layers.Input(shape=(im_height, im_width, 3), dtype="float32")
    # (1)第一模塊 :
    # (None, 224, 224, 3)
    x = layers.Conv2D(64, kernel_size=7, strides=2, padding="SAME", activation="relu", name="conv2d_1")(input_image)
    # (None, 112, 112, 64)
    x = layers.MaxPool2D(pool_size=3, strides=2, padding="SAME", name="maxpool_1")(x)
    # (2)第二模塊 :
    # (None, 56, 56, 64)
    x = layers.Conv2D(64, kernel_size=1, activation="relu", name="conv2d_2")(x)
    # (None, 56, 56, 64)
    x = layers.Conv2D(192, kernel_size=3, padding="SAME", activation="relu", name="conv2d_3")(x)
    # (None, 56, 56, 192)
    x = layers.MaxPool2D(pool_size=3, strides=2, padding="SAME", name="maxpool_2")(x)
       
    # (3)第三模塊 :
    # (None, 28, 28, 192)
    x = Inception(64, 96, 128, 16, 32, 32, name="inception_3a")(x)
    # (None, 28, 28, 256)
    x = Inception(128, 128, 192, 32, 96, 64, name="inception_3b")(x)
    # (None, 28, 28, 480)
    x = layers.MaxPool2D(pool_size=3, strides=2, padding="SAME", name="maxpool_3")(x)
    
    # (4)第四模塊 :
    # (None, 14, 14, 480)
    x = Inception(192, 96, 208, 16, 48, 64, name="inception_4a")(x)
    if aux_logits:
        aux1 = InceptionAux(class_num, name="aux_1")(x)

    # (None, 14, 14, 512)
    x = Inception(160, 112, 224, 24, 64, 64, name="inception_4b")(x)
    # (None, 14, 14, 512)
    x = Inception(128, 128, 256, 24, 64, 64, name="inception_4c")(x)
    # (None, 14, 14, 512)
    x = Inception(112, 144, 288, 32, 64, 64, name="inception_4d")(x)
    if aux_logits:
        aux2 = InceptionAux(class_num, name="aux_2")(x)

    # (None, 14, 14, 528)
    x = Inception(256, 160, 320, 32, 128, 128, name="inception_4e")(x)
    # (None, 14, 14, 532)
    x = layers.MaxPool2D(pool_size=3, strides=2, padding="SAME", name="maxpool_4")(x)
    
    # (5)第五模塊 :
    # (None, 7, 7, 832)
    x = Inception(256, 160, 320, 32, 128, 128, name="inception_5a")(x)
    # (None, 7, 7, 832)
    x = Inception(384, 192, 384, 48, 128, 128, name="inception_5b")(x)
    # (None, 7, 7, 1024)
    x = layers.AvgPool2D(pool_size=7, strides=1, name="avgpool_1")(x)
    
    # (6)最後輸出 :
    # (None, 1, 1, 1024)
    x = layers.Flatten(name="output_flatten")(x)
    # (None, 1024)
    x = layers.Dropout(rate=0.4, name="output_dropout")(x)
    x = layers.Dense(class_num, name="output_dense")(x)
    # (None, class_num)
    aux3 = layers.Softmax(name="aux_3")(x)

    if aux_logits:
        model = models.Model(inputs=input_image, outputs=[aux1, aux2, aux3])
    else:
        model = models.Model(inputs=input_image, outputs=aux3)
    return model

In [18]:
im_height = 224
im_width = 224
batch_size = 32
model = GoogLeNet(im_height=im_height, im_width=im_width, class_num=10, aux_logits=True)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 112, 112, 64) 9472        input_3[0][0]                    
__________________________________________________________________________________________________
maxpool_1 (MaxPooling2D)        (None, 56, 56, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 56, 56, 64)   4160        maxpool_1[0][0]                  
____________________________________________________________________________________________

In [19]:
import tensorflow as tf
# 使用 keras 所提供的優化函數與損失函數
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0003)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
# 定義訓練階段函數
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        aux1, aux2, output = model(images, training=True)
        loss1 = loss_object(labels, aux1)
        loss2 = loss_object(labels, aux2)
        loss3 = loss_object(labels, output)
        loss = loss1 * 0.3 + loss2 * 0.3 + loss3
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # 計算訓練的效能衡量指標
    train_loss(loss)
    train_accuracy(labels, output)
# 定義驗證階段函數   
@tf.function
def val_step(images, labels):
    _, _, output = model(images, training=False)
    loss = loss_object(labels, output)
    val_loss(loss)
    val_accuracy(labels, output)

In [20]:
from tensorflow.keras.datasets import cifar10
import tensorflow as tf
# 載入 cifar10 資料集 50000張訓練資料 , 10000張測試資料, 每張大小為 32x32,3通道
(train_Data, train_Label), (test_Data, test_Label) = cifar10.load_data()

# 資料切割, 訓練資料的前面 5000 筆當作是驗證集, 剩下的為測試集
validation_data, validation_label = train_Data[:5000],train_Label[:5000]
train_Data,train_Label= train_Data[5000:],train_Label[5000:]

batchs = 32
resize = 224

train_ds = tf.data.Dataset.from_tensor_slices((train_Data,train_Label))
test_ds = tf.data.Dataset.from_tensor_slices((test_Data, test_Label))
validation_ds = tf.data.Dataset.from_tensor_slices((validation_data,
                                                    validation_label))

def preprocess(image, label):
    image = tf.image.per_image_standardization(image)
    image = tf.image.resize(image,(resize,resize))
    return image,label

batch_size = 360
train_ds = train_ds.map(preprocess).shuffle(1000).batch(batch_size=batch_size)
# 驗證集與測試集資料不用打散
validation_ds = validation_ds.map(preprocess).batch(batch_size=batch_size)
test_ds = test_ds.map(preprocess).batch(batch_size=batch_size)


In [21]:
epochs = 10
for epoch in range(epochs):
    train_loss.reset_states()  # clear history info
    train_accuracy.reset_states()  # clear history info
    val_loss.reset_states()  # clear history info
    val_accuracy.reset_states()  # clear history info
    print("epoch = ",epoch + 1)
    for step,(batch_data,batch_label) in enumerate(train_ds):
        train_step(batch_data,batch_label)
        print("train_accuracy = ",train_accuracy.result())
        print("train_loss = ",train_loss.result())
    
    for step,(batch_data,batch_label) in enumerate(test_ds):
        val_step(batch_data, batch_label)
    # 每一次 epoch 印出驗證集的正確率與損失率
    print("val_accuracy = ",val_accuracy.result())
    print("val_loss = ",val_loss.result())

epoch =  1
train_accuracy =  tf.Tensor(0.10555556, shape=(), dtype=float32)
train_loss =  tf.Tensor(11.136734, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.08611111, shape=(), dtype=float32)
train_loss =  tf.Tensor(8.653922, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.09351852, shape=(), dtype=float32)
train_loss =  tf.Tensor(7.2649045, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.093055554, shape=(), dtype=float32)
train_loss =  tf.Tensor(6.4348106, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.09611111, shape=(), dtype=float32)
train_loss =  tf.Tensor(5.901975, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.09490741, shape=(), dtype=float32)
train_loss =  tf.Tensor(5.538494, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.0984127, shape=(), dtype=float32)
train_loss =  tf.Tensor(5.2751727, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.099305555, shape=(), dtype=float32)
train_loss =  tf.Tensor(5.0756993, shape=(), dtype=

train_accuracy =  tf.Tensor(0.20559701, shape=(), dtype=float32)
train_loss =  tf.Tensor(3.480116, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.2061683, shape=(), dtype=float32)
train_loss =  tf.Tensor(3.473939, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.20756844, shape=(), dtype=float32)
train_loss =  tf.Tensor(3.465757, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.20849207, shape=(), dtype=float32)
train_loss =  tf.Tensor(3.460566, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.20974179, shape=(), dtype=float32)
train_loss =  tf.Tensor(3.4539351, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.20999229, shape=(), dtype=float32)
train_loss =  tf.Tensor(3.448098, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.21149163, shape=(), dtype=float32)
train_loss =  tf.Tensor(3.4405892, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.21231231, shape=(), dtype=float32)
train_loss =  tf.Tensor(3.4364634, shape=(), dtype=float32)
train_

train_accuracy =  tf.Tensor(0.35912699, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.7689793, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.3548611, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.7730637, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.35462964, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.7682958, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.35694444, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.758914, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.3560606, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.756753, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.35509259, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.7570484, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.3542735, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.7529476, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.3513889, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.7522793, shape=(), dtype=float32)
train_

train_accuracy =  tf.Tensor(0.37241247, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.656771, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.37304804, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.6551902, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.37322223, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.6548898, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.37320906, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.6551995, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.3733045, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.6547148, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.37343305, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.6537113, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.37443742, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.6503448, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.37538195, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.6472223, shape=(), dtype=float32)
tr

train_accuracy =  tf.Tensor(0.45833334, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.3635216, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.4579365, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.366728, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.4564815, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.366686, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.45868057, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.3682282, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.45898694, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.3707042, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.45802468, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.3700337, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.45643276, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.3732123, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.45805556, shape=(), dtype=float32)
train_loss =  tf.Tensor(2.3658404, shape=(), dtype=float32)
trai

...

train_accuracy =  tf.Tensor(0.6865698, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.4925394, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.68651515, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.4920425, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.6869369, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.4909343, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.6869544, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.4907767, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.687291, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.4896659, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.6876706, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.4879379, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.68753624, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.4880644, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.68778735, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.4873595, shape=(), dtype=float32)
train_

train_accuracy =  tf.Tensor(0.7126984, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.3860728, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.71316665, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.3835382, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.7133442, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.3819165, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.7133013, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.3812282, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.713522, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.3802756, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.71332306, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.3813514, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.7134848, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.3790772, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.7139385, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.3765357, shape=(), dtype=float32)
train_a

train_accuracy =  tf.Tensor(0.7200483, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.3556052, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.72018677, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.3550622, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.7200143, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.3554606, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.7202213, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.3547826, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.7205649, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.3538934, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.72078705, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.3533865, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.72098255, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.3523715, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.7211065, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.3519278, shape=(), dtype=float32)
train

train_accuracy =  tf.Tensor(0.7410606, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.2820188, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.74151784, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.2799872, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.7417154, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.278341, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.7414751, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.2760712, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.74124295, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.2760222, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.7410648, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.2785171, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.7412113, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.2780968, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.74148744, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.2772853, shape=(), dtype=float32)
train_

train_accuracy =  tf.Tensor(0.73987603, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.2811112, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.7400273, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.2804673, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.7402891, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.2796357, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.740681, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.2782829, shape=(), dtype=float32)
train_accuracy =  tf.Tensor(0.7407333, shape=(), dtype=float32)
train_loss =  tf.Tensor(1.2783424, shape=(), dtype=float32)
val_accuracy =  tf.Tensor(0.6793, shape=(), dtype=float32)
val_loss =  tf.Tensor(0.95787483, shape=(), dtype=float32)
